In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pickle
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier   
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from mlxtend.feature_selection import SequentialFeatureSelector as sfs


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def sfsFeature(indep_x,dep_y):
    SFSlist=[]
    SFSlist1=[]
    SFSlist2=[]
     


    log_model = LogisticRegression(solver='lbfgs')
    #svc_model=SVC(kernel='linear',max_iter=100,random_state=0)
    #DT=DecisionTreeClassifier(criterion='gini',max_features='sqrt',splitter='best',random_state=0)
    #RF=RandomForestClassifier(n_estimators=10,criterion='entropy',random_state=0)
    SFSmodellist=[log_model]
    

    for model in SFSmodellist:
        log_sfs = sfs(model, k_features=6, forward=True, floating=False, verbose=0, scoring='accuracy', cv=5, n_jobs=1)
        log_fit = log_sfs.fit(indep_x, dep_y)
        log_sfs_feature = log_fit.transform(indep_x)
        SFSlist.append(log_sfs_feature)

        # Get the indices and names of selected features
        op = log_sfs.k_feature_idx_
        op1 = log_sfs.k_feature_names_
        SFSlist1.append(op)
        SFSlist2.append(op1)

    return SFSlist, SFSlist1, SFSlist2

In [4]:
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        
        #Feature Scaling
        #from sklearn.preprocessing import StandardScaler
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)
        
        return X_train, X_test, y_train, y_test, sc
    
def cm_prediction(classifier,X_test):
        y_pred = classifier.predict(X_test)
        
        # Making the Confusion Matrix
        from sklearn.metrics import confusion_matrix
        cm = confusion_matrix(y_test, y_pred)
        
        from sklearn.metrics import accuracy_score 
        from sklearn.metrics import classification_report 
        
        Accuracy=accuracy_score(y_test, y_pred )
        
        report=classification_report(y_test, y_pred)
        return  classifier,Accuracy,report,X_test,y_test,cm


In [5]:
def svm_NL(X_train,y_train,X_test):
                
        from sklearn.svm import SVC
        classifier = SVC(kernel = 'rbf', random_state = 0)
        classifier.fit(X_train, y_train)
        classifier,Accuracy,report,X_test,y_test,cm=cm_prediction(classifier,X_test)
        return  classifier,Accuracy,report,X_test,y_test,cm



def sfs_classification(accsvmnl):
   
    sfsdataframe = pd.DataFrame(index=['Logistic'],
                                 columns=[ 'SVMnl'])
    for number,idex in enumerate(sfsdataframe.index):
        sfsdataframe['SVMnl'][idex]=accsvmnl[number]
    return sfsdataframe

In [6]:
dataset1=pd.read_csv("prep.csv",index_col=None)
df2=dataset1
df2 = pd.get_dummies(df2, drop_first=True)

indep_X=df2.drop('classification_yes', axis=1)
dep_Y=df2['classification_yes']

In [7]:
!pip install --upgrade mlxtend

In [10]:
SFSlist=sfsFeature(indep_X,dep_Y)       


accsvmnl=[]


In [11]:
SFSlist, SFSlist1,SFSlist2=SFSlist

In [12]:
SFSlist

[array([[ 3.        ,  0.        , 12.51815562, 38.86890244,  0.        ,
          0.        ],
        [ 2.        ,  0.        , 10.7       , 34.        ,  0.        ,
          1.        ],
        [ 1.        ,  0.        , 12.        , 34.        ,  0.        ,
          1.        ],
        ...,
        [ 3.        ,  0.        ,  9.1       , 26.        ,  0.        ,
          1.        ],
        [ 0.        ,  0.        ,  8.5       , 38.86890244,  0.        ,
          1.        ],
        [ 0.        ,  0.        , 16.3       , 53.        ,  0.        ,
          1.        ]])]

In [13]:
 SFSlist1

[(2, 3, 9, 10, 15, 18)]

In [14]:
SFSlist2

[('al', 'su', 'hrmo', 'pcv', 'sg_d', 'pc_normal')]

In [15]:
for i in SFSlist:   
    

    X_train, X_test, y_train, y_test,sc=split_scalar(i,dep_Y)   
    
    
    classifier,Accuracy,report,X_test,y_test,cm=svm_NL(X_train,y_train,X_test)  
    accsvmnl.append(Accuracy)
    
    
result=sfs_classification(accsvmnl)

In [16]:
result
#6

,SVMnl
Logistic,0.99


In [17]:
import pickle as pkl
filename="finalized_model_Logistic.sav"

In [18]:
pickle.dump(classifier,open(filename,'wb'))

In [19]:
filenamex="sc.pkl"

In [20]:
pickle.dump(sc,open(filenamex,'wb'))

In [21]:
sc=pickle.load(open("sc.pkl",'rb'))

In [22]:
preinput = sc.transform([[0,0,16.7 ,50,0,1]])
preinput

array([[-0.65587132, -0.36317599,  1.55577448,  1.34716496, -0.46416285,
         0.49581583]])

In [23]:
loaded_model=pickle.load(open("finalized_model_Logistic.sav",'rb')) 
result = loaded_model.predict(preinput)

In [24]:
result

array([0], dtype=uint8)